In [ ]:
## 案例4：使用梯形法求PI值

In [ ]:
"""
对半径为R的四分之一圆,我们可以采用微分方法依次排列小矩形(类似于梯形台阶)。当矩形数量达到无穷大的
N时,矩形的总面积就接近于真实的1/4圆面积。在以下示例中，我们设置R=1、N=64*1024*1024。假设此时有
size个进程参与计算,首先计算每个进程需要处理的矩形数量(N//size)。接下来，后size-1个进程作为Worker，
计算各自矩形面积之和发送给主进程。而第1个进程作为Master，接收各Worker发送数据，汇总所有矩形面积，
从而近似计算出PI值。
"""

In [ ]:
from mpi4py import MPI
import time
import math
communicator=MPI.COMM_WORLD
rank=communicator.Get_rank()  #进程唯一的标识Rank
process_nums=communicator.Get_size()
t0=time.time()
rect_num=64 * 1024 * 1024
rect_width=1/rect_num
step_size=rect_num//process_nums

def cal_rect_area(process_no,step_size,rect_width):
    tot_area=0.0
    rect_start=(process_no*step_size+1)*rect_width
    for i in range(step_size):
        x=rect_start+i*rect_width
        #  (x,y) 对应于第i个小矩形唯一在圆弧上的顶点，容易知道x^2+y^2=1  ==> y=sqrt(1-x^2)
        rect_length=math.pow(1-x*x, 0.5)
        tot_area+=rect_width*rect_length
    return tot_area


tot_area=cal_rect_area(rank,step_size,rect_width)
if rank==0:
    # Master 进程
    for i in range(1,process_nums):
        tot_area+=communicator.recv(source=i)
    p_i=tot_area * 4
    t1=time.time()
    print('模拟PI值为: {:.10f}, 相对误差为：{:.10f}'.format(p_i,abs(1-p_i/math.pi))) 
    print('并行耗时：{:.3f}s'.format(t1 - t0))
else:
    # Worker 进程
    communicator.send(tot_area,dest=0)